In [12]:
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as random
from scipy.signal import convolve
import scipy
from joblib import Parallel, delayed
import time
from scipy import sparse
from tqdm import tqdm, tqdm_notebook

%run ../Scripts/coverage.py
%run ../Scripts/fitness.py
%run ../Scripts/immunity.py
%run ../Scripts/initMethods.py
%run ../Scripts/makeGif.py
%run ../Scripts/mutation.py

In [9]:
params = { #parameters relevant for the equations
    "Nh":        10000000,
    "N0":          100000,
    "R0":              20,
    "M":                1, #Also L, total number of spacers
    "D":                3, #Unused
    "mu":              10, #mutation rate
    "v0":              10,
    "gamma_shape":     20, 
    "Np":               0, #Number of Cas Protein
    "dc":               3, #Required number of complexes to activate defence
    "h":                4, #coordination coeff
    "r":               10, #cross-reactivity kernel
    "rho":            5e-5, #spacer sharing coeff
}
sim_params = { #parameters relevant for the simulation (including Inital Valuess)
    "xdomain":                  10000,
    "dx":                           1,
    "t0":                           0, 
    "tf":                        1000,
    "dt":                           1,
    "initial_mean_n":           [0,0],
    "initial_var_n":               10,
    "initial_mean_nh":          [0,-40],
    "initial_var_nh":              10,
    "n_step_prior":                 5,
    "conv_size":                  100,
    "num_threads":                 32,
    "tail_axis":               [1, 1],
}

In [14]:
R0_sweep = [20]
kernel = init_quarter_kernel(params, sim_params)
foldername = "test0/"
write2json(foldername, params, sim_params)

for R0 in R0_sweep:
    params["R0"] = R0
    st0 = time.time()
    n = init_guassian(params["N0"], sim_params, "n")
    # nh = init_exptail_parallel(params["Nh"], params,sim_params)
    nh = init_guassian(params["Nh"], sim_params, "nh")
    st1 = time.time()

    et0 = time_conv(st1-st0)
    print(f"Reproduction Rate R0: {R0} | init time: {et0}")

    for i in range(10000):

        if (i%100 == 0 or i < 20):
            sparse.save_npz(foldername+f"sp_frame_n{i}",n.tocoo())
            sparse.save_npz(foldername+f"sp_frame_nh{i}",nh.tocoo())
            print("Current Time Done: ", i, "|  Time Used: ", time_conv(st1-st0))

        st1 = time.time()
        p = coverage_sparse_parrallel(nh, n, kernel, params, sim_params)
        st2 = time.time()
        f = fitness_spacers(n, nh, p, params, sim_params) #f is now a masked array (where mask is where eff_R0 = 0)
        st3 = time.time()
        f_avg = np.sum(f*n/np.sum(n))
        n = virus_growth(n, f, params, sim_params) #update
        st4 = time.time()

        et1 = time.strftime("%H:%M:%S", time.gmtime(st2 - st1))
        et2 = time.strftime("%H:%M:%S", time.gmtime(st3 - st2))
        et3 = time.strftime("%H:%M:%S", time.gmtime(st4 - st3))
        print("Convolution Time:", et1, "| Fitness Spacer Time", et2, "| Virus Growth Time", et3)

        n = mutation(n, params, sim_params)
        st5 = time.time()
        nh = immunity_update(nh, n, params, sim_params) #update nh
        st6 = time.time()

        et4 = time.strftime("%H:%M:%S", time.gmtime(st5 - st4))
        et5 = time.strftime("%H:%M:%S", time.gmtime(st6 - st5))
        print("Mutation Parallel:", et4, "| Immunity Update Parallel:", et5, "| Number of Phage:", np.sum(n))

        if np.sum(n) >= 10*params["Nh"]:
            break